In [ ]:
DATA="../../cocUptoJuly2016.csv"
library(dplyr)
library(lubridate)
library(xts)

In [ ]:
raw <- read.csv(DATA, stringsAsFactors = F)
head(raw)
raw$Complaint.Date <- as.POSIXct(raw$Complaint.Date, format = "%m/%d/%Y %H:%M:%S")

In [ ]:
raw <- select(raw, Ward, Complaint.Type, Complaint.Date)

In [ ]:
complaints.data <- raw[raw$Complaint.Date >= strptime("01/01/2012 00:00:00", format = "%m/%d/%Y %H:%M:%S"), ] 
complaints.data$NumComplaints <- 1

In [ ]:
min.date <- min(complaints.data$Complaint.Date)
max.date <- max(complaints.data$Complaint.Date)

In [ ]:
periodicity_ <- function(complaints.frame, periodicity) {
    stopifnot(nrow(complaints.frame) > 0)
    series <- xts(complaints.frame$NumComplaints, complaints.frame$Complaint.Date)
    if(periodicity == "hour") {
        series <- period.apply(series, endpoints(series, "hours"), FUN = sum)
    } else if (periodicity == "day") {
        series <- apply.daily(series, FUN = sum)
    }
    series
} 

In [ ]:
# construct city level data
city.level.data <- function(periodicity) {
    periodicity_(complaints.data, periodicity)
}
plot(city.level.data("hour"))

In [ ]:
plot(city.level.data("day"))

In [ ]:
# ward level data
ward.level.data <- function(ward, periodicity) {
    df <- filter(complaints.data, Ward == ward)
    periodicity_(df, periodicity)
}
plot(ward.level.data("N188", "hour"))

In [ ]:
plot(ward.level.data("N188", "day"))

In [ ]:
# complaint level of data
complaint.level.data <- function(complaint.type, periodicity) {
    df <- filter(complaints.data, Complaint.Type == complaint.type)
    periodicity_(df, periodicity)
}
plot(complaint.level.data("Mosquito menace ", "hour"))

In [ ]:
plot(complaint.level.data("Mosquito menace ", "day"))

In [ ]:
# the above functions are stored in lib/alerts_data.R